In [1]:
#!git clone https://github.com/fchollet/keras.git && cd keras && python setup.py install --user

In [2]:
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K 
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
import numpy as np
import os

Using TensorFlow backend.
/Users/ewallner/.brew/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
num_words = 435
max_caption_len = 50
tokenizer = Tokenizer(num_words=num_words, filters='', split=" ", lower=False)

def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

X = []
for filename in os.listdir('html/'):
    X.append(load_doc('html/'+filename))

images = []
for filename in os.listdir('images/'):
    images.append(img_to_array(load_img('images/'+filename, target_size=(299, 299))))
images = np.array(images, dtype=float)
images = preprocess_input(images)
    
tokenizer.fit_on_texts(X)

vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(X)
max_length = max(len(s) for s in sequences)
 
X, y, image_data = list(), list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        image_data.append(images[img_no])
        X.append(in_seq[-50:])
        y.append(out_seq)

X, y, image_data = np.array(X), np.array(y), np.array(image_data)

In [4]:
print(image_data.shape)
print(X.shape)
print(y.shape)

(2305, 299, 299, 3)
(2305, 50)
(2305, 436)


In [5]:
IR2 = InceptionResNetV2(weights=None, include_top=True)
ir2_out = Dense(1024, activation='relu')(IR2.output)
ir2_out = RepeatVector(max_caption_len)(ir2_out)

language_input = Input(shape=(max_caption_len,))
language_model = Embedding(vocab_size, 32, input_length=max_caption_len)(language_input)
print(language_model.shape)
language_model = LSTM(128, return_sequences=True)(language_model)
print(language_model.shape)
language_model = TimeDistributed(Dense(128, activation='relu'))(language_model)
print(language_model.shape)

decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=False)(decoder)
print(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[IR2.input, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

(?, 50, 32)
(?, ?, 128)
(?, 50, 128)
Tensor("lstm_2/TensorArrayReadV3:0", shape=(?, 512), dtype=float32)


In [6]:
model.fit([image_data, X], y, batch_size=1, shuffle=False, epochs=5)

Epoch 1/5
 174/2305 [=>............................] - ETA: 1:52:34 - loss: 5.9834

KeyboardInterrupt: 